In [3]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import confusion_matrix, classification_report
from tkinter import * # __all__
from tkinter import filedialog
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from time import gmtime, strftime, localtime
import sklearn.neighbors._base

sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from flaml import AutoML

import xlwings as xw
import pandas as pd
import numpy as np
import os, re, flaml
import tkinter.ttk as ttk
import tkinter.messagebox as msgbox

def total_predict(dict_, train_x, train_y, test_x, clf_opt = True):
    pred_dict = {}
    cnt = 0
    
    for name, model in dict_.items():
        model = dict_[name]
        model.fit(train_x, train_y)
        predict_y = model.predict(test_x)
        pred_dict[name] = predict_y

        ## get cv score
        if clf_opt:
            clf_5 = cross_validate(model, train_x, train_y, cv = 5, scoring=['accuracy', 'f1', 'roc_auc'])
            save_df = pd.DataFrame({
                        "정확도": [np.round(clf_5['test_accuracy'].mean(), 4)],
                        "F1 Score": [np.round(clf_5['test_f1'].mean(), 4)],
                        "ROC-AUC": [np.round(clf_5['test_roc_auc'].mean(), 4)]})
            
        else:
            reg_5 = cross_validate(model, train_x, train_y, cv = 5, scoring=['r2', 'neg_mean_absolute_error'])
            save_df = pd.DataFrame({
                        "R-Square": [np.round(reg_5['test_r2'].mean(),4)],
                        "MAE": [-np.round(reg_5['test_neg_mean_absolute_error'].mean(),4)]})

        if cnt == 0 :
            cv_results = save_df
        else :
            cv_results = cv_results.append(save_df)

        cnt += 1

    results_df = pd.DataFrame.from_dict(pred_dict)
    
    return results_df, cv_results

def check_error(train_data, test_data, index_col, target_col):
    error_cond = False
    index_ = False
    
    try:
        if index_col != "":
            train_data.loc[:, [index_col]]
            test_data.loc[:, [index_col]]
            index_ = True
    except:
        msgbox.showerror("에러", "올바른 기준열을 입력하세요.")
        error_cond = True
    
    # 1. target col이 잘못됐을 경우 에러 메시지 팝업
    try:
        train_data.loc[:, str(target_col)]

    except KeyError:
        msgbox.showerror("에러", "올바른 타겟명을 입력하세요.")
        error_cond = True
    
    return error_cond, index_

# load data
def main_functions():
    ## 저장경로 에러
    if txt_dest_path.get() == "":
        msgbox.showerror("에러", "저장 경로를 선택하세요.")
    else:
        ## load files
        # 예외 처리 필요, 여러 개일 경우 행으로 병합
        # 우선은 단일 데이터만 돌아가게 수정
        train_datas = []
        test_datas = []

        for parent in list_file.get_children():
            train_datas.append(read_xlsx(list_file.item(parent)["values"][1]))

        for parent in list_file2.get_children():
            test_datas.append(read_xlsx(list_file2.item(parent)["values"][1]))

        if len(train_datas) == 1:
            train_data = train_datas[0]

        if len(test_datas) == 1:
            test_data = test_datas[0]

        target_col = target_space.get()
        target_col = re.sub("\n","",target_col)

        except_cols = except_space.get()
        except_cols = re.sub("\n","",except_cols)
        
        if except_cols != "":
            except_cols = [re.sub("\n","",x) for x in except_cols.split(",")]
            except_cols = [x[1:] if x[0] == " " else x for x in except_cols]
            except_cols = [x[:len(x)] if x[len(x)-1] == " " else x for x in except_cols]
    
        index_col = index_space.get()
        index_col = re.sub("\n","",index_col)
        
        # error check
        error_cond, index_ = check_error(train_data, test_data, index_col, target_col)
        
        if index_:
            train_index = train_data.loc[:, [index_col]]
            test_index = test_data.loc[:, [index_col]]

        if except_cols != "":        
            train_data = train_data.loc[:, ~train_data.columns.isin(except_cols)]
            test_data = test_data.loc[:, ~test_data.columns.isin(except_cols)]
        
        # error가 없으면 아래 내용 진행
        if not error_cond:
            # zero imputation
            train_data.replace(np.NaN, 0, inplace = True)
            test_data.replace(np.NaN, 0, inplace = True)

            # 기본 SETTING
            cate_dict = {"분류 예측": 'classification',
                         '수치 예측':'regression'}

            #use_name = ['LightGBM', 'XGBoost', 'CatBoost', 'RandomForest']
            #hpo_name = ['lgbm', 'xgboost', 'catboost', 'rf']

            #use_name = ['LightGBM', 'CatBoost', 'RandomForest']
            #hpo_name = ['lgbm', 'catboost', 'rf']
            
            use_name = ['LightGBM', 'CatBoost']
            hpo_name = ['lgbm', 'catboost']

            hpo_dict = {x:y for x,y in zip(use_name, hpo_name)}

            algo_name = ml_algo.get()
            cate_name = ml_cate.get()

            tune_condition = (current_value.get() != 0) and (radio_var.get() == 1) and (algo_name not in ['전체', 'SVM', 'Logistic Regression', 'SVR', 'ElasticNet', 'Linear Regression', 'RandomForest'])

            ## training models
            if cate_name == "분류 예측":
                change_cols = False
                # target column 데이터 타입 체크 후 텍스트 데이터일 때
                if type(train_data[target_col][0]) == str:
                    mapping_cols1 = {x:y for x,y in zip(np.unique(train_data[target_col]), range(np.unique(train_data[target_col]).shape[0]))}
                    mapping_cols2 = {y:x for x,y in zip(np.unique(train_data[target_col]), range(np.unique(train_data[target_col]).shape[0]))}

                    train_data[target_col] = train_data[target_col].map(mapping_cols1)

                    # 복원용 함수
                    def custom_map(x):
                        return x.map(mapping_cols2)

                    change_cols = True

                # data setting
                train_x, train_y, test_x, col_names = data_split(train_data, test_data, target_col, clf_opt = True)

                # set model
                try:
                    if algo_name == '전체':
                        results_df, cv_results = total_predict(clf_dict, train_x, train_y, test_x, clf_opt = True)
                        clf_range = len(clf_dict.keys())

                        # target명 복원
                        if change_cols:
                            results_df = results_df.apply(custom_map, axis = 1)

                        append_df1 = pd.DataFrame({" ": [" " for x in range(clf_range)],
                                                   "  ": ["  " for x in range(clf_range)],
                                                   "모델명": [x for x in clf_dict.keys()]})

                        cv_results.index = [x for x in range(cv_results.shape[0])]
                        
                        if index_:
                            results_df = pd.concat([test_index, results_df], axis = 1)
                        append_df1 = pd.concat([append_df1, cv_results], axis = 1)
                        results_df = pd.concat([results_df, append_df1], axis = 1)

                        algo_name = "Total"

                    else:
                        # 하이퍼파라미텨 튜닝 yes
                        if tune_condition:
                            model = AutoML()
                            optim_nums = current_value.get()
                            hpo_settings = {
                                "task": cate_dict[cate_name],
                                "estimator_list":[hpo_dict[algo_name]],
                                "max_iter": optim_nums,
                                "eval_method": "cv",
                                "n_splits": 5,
                                'verbose': 0
                            }

                            model.fit(train_x, train_y, **hpo_settings)
                            best_params = model.best_config
                            predict_y = model.predict(test_x)
                            msgbox.showinfo("최적 하이퍼 파라미터", best_params)

                            model_cv = model.model

                        else:
                            model = clf_dict[algo_name]
                            model.fit(train_x, train_y)
                            predict_y = model.predict(test_x)

                            model_cv = model

                        results_df = pd.DataFrame(predict_y)
                        results_df.columns = [algo_name]

                        # 타겟명 복원
                        if change_cols:
                            results_df[algo_name] = results_df[algo_name].map(mapping_cols2)

                        # get cross validation score
                        clf_5 = cross_validate(model_cv, train_x, train_y, cv = 5, scoring=['accuracy', 'f1', 'roc_auc'])
                        perform_length = len(clf_5.keys()) - 2
                        perform_df = pd.DataFrame({"  ": [" " for _ in range(perform_length)], # 결합 시 공백
                                                 "   ": ["  " for _ in range(perform_length)], # 결합 시 공백
                                                  '측정 지표': ["정확도", "F1 Score", "ROC-AUC"],
                                                  "성능": [np.round(clf_5['test_accuracy'].mean(),4),
                                                           np.round(clf_5['test_f1'].mean(),4),
                                                           np.round(clf_5['test_roc_auc'].mean(),4)]})
                        if index_:
                            results_df = pd.concat([test_index, results_df], axis = 1)
                        results_df = pd.concat([results_df, perform_df], axis = 1)

                except:
                    msgbox.showerror("에러", "모델링 에러, 인풋 데이터를 재확인하세요")

            elif cate_name == "수치 예측":
                # target column 데이터 타입 체크
                if type(train_data[target_col][0]) not in [float, int]:
                    try:
                        train_data[target_col] = [float(x) for x in train_data[target_col]]
                    except ValueError:
                        msgbox.showerror("에러", "타겟 열이 수치형 데이터가 아닙니다.")

                # data setting
                train_x, train_y, test_x, col_names = data_split(train_data, test_data, target_col)

                try:
                    if algo_name == '전체':
                        results_df, cv_results = total_predict(reg_dict, train_x, train_y, test_x, clf_opt = False)
                        reg_range = len(reg_dict.keys())

                        append_df1 = pd.DataFrame({" ": [" " for x in range(reg_range)],
                                                   "  ": ["  " for x in range(reg_range)],
                                                   "모델명": [x for x in reg_dict.keys()]})

                        cv_results.index = [x for x in range(cv_results.shape[0])]
                        
                        if index_:
                            results_df = pd.concat([test_index, results_df], axis = 1)
                        append_df1 = pd.concat([append_df1, cv_results], axis = 1)
                        results_df = pd.concat([results_df, append_df1], axis = 1)

                        algo_name = "Total"

                    else:
                        # 하이퍼파라미텨 튜닝 yes
                        if tune_condition:
                            model = AutoML()
                            optim_nums = current_value.get()
                            hpo_settings = {
                                "task": cate_dict[cate_name],
                                "estimator_list":[hpo_dict[algo_name]],
                                "max_iter": optim_nums,
                                "eval_method": "cv",
                                "n_splits": 5,
                                'verbose': 0
                            }

                            model.fit(train_x, train_y, **hpo_settings)
                            best_params = model.best_config
                            predict_y = model.predict(test_x)
                            msgbox.showinfo("최적 하이퍼 파라미터", best_params)

                            model_cv = model.model

                        else:
                            model = reg_dict[algo_name]
                            model.fit(train_x, train_y)
                            predict_y = model.predict(test_x)

                            model_cv = model

                        results_df = pd.DataFrame(predict_y)
                        results_df.columns = [algo_name]

                        # get cross validation score
                        reg_5 = cross_validate(model_cv, train_x, train_y, cv = 5, scoring=['r2', 'neg_mean_absolute_error'])
                        perform_length = len(reg_5.keys()) - 2
                        perform_df = pd.DataFrame({"  ": [" " for _ in range(perform_length)],
                                                "   ": ["  " for _ in range(perform_length)],
                                              '측정 지표': ["R-Square", "MAE"],
                                              "성능": [np.round(reg_5['test_r2'].mean(),4),
                                                       -np.round(reg_5['test_neg_mean_absolute_error'].mean(),4)]})
                        
                        if index_:
                            results_df = pd.concat([test_index, results_df], axis = 1)
                        results_df = pd.concat([results_df, perform_df], axis = 1)

                except:
                    msgbox.showerror("에러", "모델링 에러, 인풋 데이터를 재확인하세요")

            elif ml_cate.get() == "시계열 예측":
                print(ml_cate.get())
            elif ml_cate.get() == "군집 분석":
                print(ml_cate.get())

            save_dict = {"분류 예측": "CLF",
                        "수치 예측": "REG"}


            dest_path = os.path.join(txt_dest_path.get(), "{type_}_{model}_results_{date}.csv".format(type_ = save_dict[cate_name], 
                                                                                                      model = algo_name,
                                                                                                      date= strftime("%Y-%m-%d %Hh%Mm%Ss", localtime())))
            results_df.to_csv(dest_path, index = False, encoding = "cp949")
            msgbox.showinfo("알림", "작업이 완료되었습니다.")

# 학습 데이터 파일 추가
def add_file():
    files = filedialog.askopenfilenames(title="데이터 선택", \
        filetypes=(("모든 파일", "*.*"),
                ("엑셀 파일", "*.xlsx"),
                ("csv 파일", "*.csv")),\
        initialdir=r"C:\python")
        # 최초에 사용자가 지정한 경로를 보여줌
    
    # 사용자가 선택한 파일 목록
    for idx, file in enumerate(files):
        list_file.insert(parent="", index = "end", iid = idx,text = file, value = (END, file))
        
# 선택 삭제
def del_file():
    selected_item = list_file.selection()[0]
    list_file.delete(selected_item)
    
    '''
    for index in reversed(list_file.curselection()):
        list_file.delete(index)
    '''

# 예측 데이터 파일 추가
def add_file2():
    files2 = filedialog.askopenfilenames(title="데이터 선택", \
        filetypes=(("모든 파일", "*.*"),
                ("엑셀 파일", "*.xlsx"),
                ("csv 파일", "*.csv")),\
        initialdir=r"C:\python")
        # 최초에 사용자가 지정한 경로를 보여줌
    
    # 사용자가 선택한 파일 목록
    for idx, file in enumerate(files2):
        list_file2.insert(parent="", index = "end", iid = idx,text = file, value = (END, file))
    
         
# 선택 삭제
def del_file2():
    selected_item = list_file2.selection()[0]
    list_file2.delete(selected_item)

    '''
    for index in reversed(list_file2.curselection()):
        list_file2.delete(index)
    '''

# 저장 경로 (폴더)
def browse_dest_path():
    folder_selected = filedialog.askdirectory()
    if folder_selected is None: # 사용자가 취소를 누를 때
        return
    txt_dest_path.delete(0, END)
    txt_dest_path.insert(0, folder_selected)

def read_xlsx(name):
    instance = xw.App(visible=False)
    xlsx_data = xw.Book(name).sheets[0]
    df = xlsx_data.range('A1').options(pd.DataFrame, index = False, expand = 'table').value
    instance.quit()
    instance.kill()
    return df

def data_split(train_data, test_data, target_col, clf_opt = False):
    train_x = train_data.loc[:, ~train_data.columns.isin([target_col])]
    test_x = test_data.loc[:, ~test_data.columns.isin([target_col])]
    
    if clf_opt:
        train_y = np.array(train_data.loc[:, target_col], dtype = np.int32)
    else:
        train_y = np.array(train_data.loc[:, target_col])
    
    col_names = train_x.columns
    
    #return np.array(train_x), train_y, np.array(test_x), test_y, col_names
    return np.array(train_x), train_y, np.array(test_x), col_names

def callbackTarget(event):
    if ml_cate.get() == "분류 예측":
        # 타겟 컬럼명 설정
        target_space['state'] = 'normal'
        #ml_algo['values'] = ('전체', 'LightGBM', 'XGBoost', 'CatBoost', 'RandomForest', 'SVM', 'Logistic Regression')
        ml_algo['values'] = ('전체', 'LightGBM', 'CatBoost', 'RandomForest', 'SVM', 'Logistic Regression')
        
    if ml_cate.get() == "수치 예측":
        # 타겟 컬럼명 설정
        target_space['state'] = 'normal'
        #ml_algo['values'] = ('전체', 'LightGBM', 'XGBoost', 'CatBoost', 'RandomForest', 'SVR', 'ElasticNet', 'Linear Regression')
        ml_algo['values'] = ('전체', 'LightGBM', 'CatBoost', 'RandomForest', 'SVR', 'ElasticNet', 'Linear Regression')
        
    if ml_cate.get() == "시계열 예측":
        # 타겟 컬럼명 설정
        target_space['state'] = 'normal'
        
    elif ml_cate.get() == "군집 분석":
        # 타겟 컬럼명 설정
        target_space['state'] = 'disabled'        

def callbackSpin(event):
    if (ml_algo.get() in ['전체', 'SVM', 'Logistic Regression', 'SVR', 'ElasticNet', 'Linear Regression', 'RandomForest']) or (radio_var.get() == 2):
        cnt_spin.config(state=DISABLED)
    else:
        cnt_spin.config(state=NORMAL)

def radio_state():
    if (ml_algo.get() in ['전체', 'SVM', 'Logistic Regression', 'SVR', 'ElasticNet', 'Linear Regression', 'RandomForest']) or (radio_var.get() == 2):
        cnt_spin.config(state=DISABLED)
    else:
        cnt_spin.config(state=NORMAL)

'''
clf_list = ['LightGBM', 'XGBoost', 'CatBoost', 'RandomForest', 'SVM', 'Logistic Regression']
reg_list = ['LightGBM', 'XGBoost', 'CatBoost', 'RandomForest', 'SVR', 'ElasticNet', 'Linear Regression']

clf_models = [LGBMClassifier(), XGBClassifier(), CatBoostClassifier(verbose = False), 
            RandomForestClassifier(), SVC(), LogisticRegression()]
reg_models = [LGBMRegressor(), XGBRegressor(), CatBoostRegressor(verbose = False), 
            RandomForestRegressor(), SVR(), ElasticNet(), LinearRegression()]
'''

clf_list = ['LightGBM', 'CatBoost', 'RandomForest', 'SVM', 'Logistic Regression']
reg_list = ['LightGBM', 'CatBoost', 'RandomForest', 'SVR', 'ElasticNet', 'Linear Regression']

clf_models = [LGBMClassifier(), CatBoostClassifier(verbose = False), 
            RandomForestClassifier(), SVC(), LogisticRegression()]
reg_models = [LGBMRegressor(), CatBoostRegressor(verbose = False), 
            RandomForestRegressor(), SVR(), ElasticNet(), LinearRegression()]

clf_dict = {x:y for x,y in zip(clf_list, clf_models)}
reg_dict = {x:y for x,y in zip(reg_list, reg_models)}

root = Tk()
root.title("GUI based AI/ML TOOL - 데이터솔루션부")
root.option_add("*tearOff", False)

# Create a style
style = ttk.Style(root)

root.tk.call("source", "./add/forest-light.tcl")
style.theme_use("forest-light")
root.update_idletasks()

# 파일 프레임 (파일 추가, 선택 삭제)
file_frame = ttk.LabelFrame(root, text = "학습 데이터")
file_frame.pack(fill="x", padx=5, pady=5) # 간격 띄우기

btn_frame = Frame(file_frame)
btn_frame.pack(side = "right")

btn_add_file = ttk.Button(btn_frame, width=10,
                      text="파일추가", command=add_file, style = "Accent.TButton")
btn_add_file.grid(row = 1, padx = 5, pady = 1, sticky = "nsew")

btn_del_file = ttk.Button(btn_frame, width=10, 
                      text="선택삭제", command=del_file)
btn_del_file.grid(row = 2, padx = 5, pady = 1, sticky = "nsew")

# 리스트 프레임
list_frame = Frame(file_frame)
list_frame.pack(fill="both", padx=5, pady=5, side = "bottom")

scrollbar = ttk.Scrollbar(list_frame)
scrollbar.pack(side="right", fill="y")

list_file = ttk.Treeview(list_frame, selectmode = "extended", yscrollcommand=scrollbar.set, height = 1)
list_file.pack(side="left", fill="both", expand=True)
scrollbar.config(command=list_file.yview)

list_file.column('#0')
list_file.heading('#0', text = "Data List", anchor = "center")

# 파일 프레임 (파일 추가, 선택 삭제)
file_frame2 = ttk.LabelFrame(root, text = "예측 데이터")
file_frame2.pack(fill="x", padx=5, pady=5) # 간격 띄우기

btn_frame2 = Frame(file_frame2)
btn_frame2.pack(side = "right")

btn_add_file2 = ttk.Button(btn_frame2, width=10,
                       text="파일추가", command=add_file2, style = "Accent.TButton")
btn_add_file2.grid(row = 1, padx = 5, pady = 1, sticky = "nsew")

btn_del_file2 = ttk.Button(btn_frame2, width=10,
                       text="선택삭제", command=del_file2)
btn_del_file2.grid(row = 2, padx = 5, pady = 1, sticky = "nsew")

# 리스트 프레임
list_frame2 = Frame(file_frame2)
list_frame2.pack(fill="both", padx=5, pady=5)

scrollbar2 = ttk.Scrollbar(list_frame2)
scrollbar2.pack(side="right", fill="y")

list_file2 = ttk.Treeview(list_frame2, selectmode = "extended", yscrollcommand=scrollbar2.set, height = 1)
list_file2.pack(side="left", fill="both", expand=True)
scrollbar2.config(command=list_file2.yview)

list_file2.column('#0')
list_file2.heading('#0', text = "Data List", anchor = "center")

# 저장 경로 프레임
path_frame = ttk.LabelFrame(root, text="저장경로")
path_frame.pack(fill="x", padx=5, pady=5, ipady=5)

txt_dest_path = ttk.Entry(path_frame)
txt_dest_path.pack(side="left", fill="x", expand=True, padx=5, pady=5, ipady=4) # 높이 변경

btn_dest_path = ttk.Button(path_frame, text="찾아보기", width=10,
                      command = browse_dest_path, style = "Accent.TButton")
btn_dest_path.pack(side="right", padx=5, pady=5)

# 옵션 프레임
frame_option = ttk.Frame(root)
frame_option.pack(padx=5, pady=5, ipady=5)

notebook = ttk.Notebook(frame_option)
ml_options = ttk.Frame(notebook)

ml_top = ttk.Frame(ml_options)
ml_bottom = ttk.Frame(ml_options)

ml_top.pack(side = "left")
ml_bottom.pack(side = "right")

# 1. 머신러닝 종류 선택 옵션
ml_cate_label = Label(ml_top, text="머신러닝 유형", width=12)
ml_cate_label.grid(row = 0, column=0, padx=2, pady=5)

opt_ml_cate = ["분류 예측", "수치 예측"]
ml_cate = ttk.Combobox(ml_top, state="readonly", values=opt_ml_cate, width=15)
ml_cate.current(0)
ml_cate.bind("<<ComboboxSelected>>", callbackTarget)
ml_cate.grid(row = 0, column=1, padx=2, pady=5)

# 2. 머신러닝 알고리즘 선택 옵션
ml_algo_label = Label(ml_bottom, text="예측 모델", width=12)
ml_algo_label.grid(row = 0, column=2, padx=2, pady=5)

#opt_ml_algo = ['전체', 'LightGBM', 'XGBoost', 'CatBoost', 'RandomForest', 'SVM', 'Logistic Regression']
opt_ml_algo = ['전체', 'LightGBM', 'CatBoost', 'RandomForest', 'SVM', 'Logistic Regression']
ml_algo = ttk.Combobox(ml_bottom, state="readonly", values=opt_ml_algo, width=15)
ml_algo.current(0)
ml_algo.bind("<<ComboboxSelected>>", callbackSpin)
ml_algo.grid(row = 0, column=3, padx=2, pady=5)

# 3. 하이퍼파라미터 튜닝 선택 옵션
radio_var = IntVar(value = 2)
radio_frame = ttk.Frame(ml_top)

ml_hpo_label = Label(ml_top, text="모델 튜닝 여부", width=15)
ml_hpo_label.grid(row = 1, column=0, padx=2, pady=5)

hpo_y = ttk.Radiobutton(radio_frame, text ="Y", variable = radio_var, value = 1, command = radio_state)
hpo_y.pack(side="left", padx=2, pady=5)

hpo_n = ttk.Radiobutton(radio_frame, text ="N", variable = radio_var, value = 2, command = radio_state)
hpo_n.pack(side="left", padx=2, pady=5)

radio_frame.grid(row=1, column=1,padx=2,pady=5)

# iteration 숫자
hpo_cnt = Label(ml_bottom, text="튜닝 횟수", width=15)
hpo_cnt.grid(row = 1, column=2, padx=2, pady=5)

current_value = IntVar(value = 0)

cnt_spin = ttk.Spinbox(ml_bottom, from_=0, to=500, width = 11, textvariable = current_value)
cnt_spin.grid(row = 1, column=3, padx=2, pady=5)

notebook.add(ml_options, text = "ML Options")

column_options = ttk.Frame(frame_option)
column_top = ttk.Frame(column_options)
column_bottom = ttk.Frame(column_options)

column_top.grid(row=0)
column_bottom.grid(row=1)

# 4. 타겟 컬럼 처리 옵션
target_format = Label(column_top, text="타겟열 이름", width=12)
target_format.grid(row=0, column =0, padx=2, pady=5)

target_space = ttk.Entry(column_top,width = 20)
target_space.grid(row=0, column=1, padx=2, pady=5)

# 5. 기준 컬럼 처리 옵션
index_format = Label(column_top, text="기준열 이름", width=12)
index_format.grid(row=0, column=2, padx=5, pady=5)

index_space = ttk.Entry(column_top, width = 20)
index_space.grid(row=0, column=3, padx=2, pady=5)

# 6. 제거 컬럼 처리 옵션
except_format = Label(column_bottom, text="제거열 이름", width=12)
except_format.grid(row=0, column=0, padx=2, pady=5)

except_space = ttk.Entry(column_bottom, width = 59)
except_space.grid(row=0, column=1, padx=2, pady=5)

notebook.add(column_options, text = "Column Options")
notebook.pack()

# 실행 프레임
frame_run = Frame(root)
frame_run.pack(fill="x", padx=5, pady=5)

btn_close = ttk.Button(frame_run, text="닫기", width=12, command=root.quit)
btn_close.pack(side="right", padx=5, pady=5)

btn_start = ttk.Button(frame_run, text="시작", width=12,
                  command = main_functions)
btn_start.pack(side="right", padx=5, pady=5)

root.resizable(True, True)
root.mainloop()